In [2]:
import json
import os

In [2]:
training_challenges_path = os.path.join("../data/arc-agi-2025/sample_augmented/arc-agi_training_challenges.json")
training_solutions_path = os.path.join("../data/arc-agi-2025/sample_augmented/arc-agi_training_solutions.json")
validation_challenges_path = os.path.join("../data/arc-agi-2025/sample_augmented/arc-agi_evaluation_challenges.json")
validation_solutions_path = os.path.join("../data/arc-agi-2025/sample_augmented/arc-agi_evaluation_solutions.json")

In [3]:
# ============================================
# ARC-AGI Prompt and Output Templates
# ============================================

# Prompt template
ARC_PROMPT_TEMPLATE = """You are given 10 example input-output grid pairs from the ARC (Abstraction and Reasoning Corpus) task.
Each grid is represented as a 2D array of integers ranging from 0 to 9. Each integer corresponds to a specific color:

0 - Black  
1 - Blue  
2 - Red  
3 - Green  
4 - Yellow  
5 - Gray  
6 - Magenta  
7 - Orange  
8 - Light Blue  
9 - Dark Red  

Each cell in the grid represents one colored square. The transformation from input to output follows a specific visual or logical pattern.

Your task:
1. Study the given 10 example input-output pairs carefully.
2. Some examples may be incorrect or noisy — identify the pattern that the *majority* of examples follow.
3. Infer the correct transformation rule that maps the input grid to the output grid.
4. Apply this inferred transformation to the provided test input grid to produce the correct output grid.

Below are the example pairs (in JSON format):

{examples}

Now, here is the test input grid (in JSON format):

{test_input}

Generate the output grid that correctly applies the inferred transformation to this test input.

Your response should be formatted strictly as a JSON code block in the following form:

```json
{{
    "output": [[...]]
}}
```"""

# Output template (for model-predicted grid)
ARC_OUTPUT_TEMPLATE = """```json
{{
    "output": {output_grid}
}}
```"""


In [4]:
import json

# Suppose you have these from your dataset
examples_json = "\n".join([
    f"Example {i+1}:\n" + json.dumps(ex, indent=4)
    for i, ex in enumerate(train_examples)
])

test_input_json = json.dumps(test_input, indent=4)

# Build the final prompt
final_prompt = ARC_PROMPT_TEMPLATE.format(
    examples=examples_json,
    test_input=test_input_json
)

# When generating the model output
output_grid = json.dumps(predicted_output, indent=4)
final_output = ARC_OUTPUT_TEMPLATE.format(output_grid=output_grid)

print(final_prompt)
print(final_output)


NameError: name 'train_examples' is not defined

In [5]:
os.makedirs("../data/arc-agi-2025/prompts", exist_ok=True)

In [6]:
training_prompts_path = os.path.join("../data/arc-agi-2025/prompts/arc-agi_training_prompts.json")
validation_prompts_path = os.path.join("../data/arc-agi-2025/prompts/arc-agi_validation_prompts.json")


In [7]:
with open(training_challenges_path, 'r') as f:
    training_challenges = json.load(f)

with open(training_solutions_path, 'r') as f:
    training_solutions = json.load(f)

with open(validation_challenges_path, 'r') as f:
    validation_challenges = json.load(f)

with open(validation_solutions_path, 'r') as f:
    validation_solutions = json.load(f)

In [16]:
def generate_prompts(challenges, solutions, save_path):
    prompts = []
    for challenge_id, challenge in challenges.items():
        solution = solutions.get(challenge_id, [])
        if solution == []:
            continue
        train_examples = challenge.get('train', [])
        test_inputs = challenge.get('test', [{}])
        if train_examples == []:
            continue
        examples_json = "\n".join([
            f"Example {i+1}:\n" + json.dumps(ex, indent=4)
            for i, ex in enumerate(train_examples)
        ])
        for i in range(len(test_inputs)):
            test_input = test_inputs[i]
            test_output = solution[i]
            if not test_input.get('input'):
                continue
            test_input_json = json.dumps(test_input.get('input', []), indent=4)
            final_prompt = ARC_PROMPT_TEMPLATE.format(
                examples=examples_json,
                test_input=test_input_json
            )
            challenge_id_new = challenge_id
            if(len(test_inputs)>1):
                challenge_id_new = f"{challenge_id}_test{i}"
            output = ARC_OUTPUT_TEMPLATE.format(output_grid=json.dumps(test_output, indent=4))
            prompts.append({
                "challenge_id": challenge_id_new,
                "prompt": final_prompt,
                "output": output
            })
    
    with open(save_path, 'w') as f:
        json.dump(prompts, f, indent=4)

In [9]:
generate_prompts(training_challenges, training_solutions, training_prompts_path)

In [10]:
generate_prompts(validation_challenges, validation_solutions, validation_prompts_path)

In [15]:
with open(training_prompts_path, 'r') as f:
    training_prompts = json.load(f)

with open(validation_prompts_path, 'r') as f:
    validation_prompts = json.load(f)

In [16]:
len(training_prompts)

1076

In [17]:
len(validation_prompts)

172

In [3]:
evaluation_challenges_path = os.path.join("../data/arc-agi-2025/raw/arc-agi_evaluation_challenges.json")
evaluation_solutions_path = os.path.join("../data/arc-agi-2025/raw/arc-agi_evaluation_solutions.json")

In [4]:
# ============================================
# ARC-AGI Prompt and Output Templates
# ============================================

# Prompt template
ARC_PROMPT_TEMPLATE = """You are given example input-output grid pairs from the ARC (Abstraction and Reasoning Corpus) task.
Each grid is represented as a 2D array of integers ranging from 0 to 9. Each integer corresponds to a specific color:

0 - Black  
1 - Blue  
2 - Red  
3 - Green  
4 - Yellow  
5 - Gray  
6 - Magenta  
7 - Orange  
8 - Light Blue  
9 - Dark Red  

Each cell in the grid represents one colored square. The transformation from input to output follows a specific visual or logical pattern.

Your task:
1. Study the given example input-output pairs carefully.
2. Some examples may be incorrect or noisy — identify the pattern that the *majority* of examples follow.
3. Infer the correct transformation rule that maps the input grid to the output grid.
4. Apply this inferred transformation to the provided test input grid to produce the correct output grid.

Below are the example pairs (in JSON format):

{examples}

Now, here is the test input grid (in JSON format):

{test_input}

Generate the output grid that correctly applies the inferred transformation to this test input.

Your response should be formatted strictly as a JSON code block in the following form:

```json
{{
    "output": [[...]]
}}
```"""

# Output template (for model-predicted grid)
ARC_OUTPUT_TEMPLATE = """```json
{{
    "output": {output_grid}
}}
```"""


In [5]:
with open(evaluation_challenges_path, 'r') as f:
    evaluation_challenges = json.load(f)


with open(evaluation_solutions_path, 'r') as f:
    evaluation_solutions = json.load(f)

In [ ]:
def generate_prompts_evaluation(challenges, solutions, save_path):
    prompts = []
    for challenge_id, challenge in challenges.items():
        solution = solutions.get(challenge_id, [])
        if solution == []:
            continue
        train_examples = challenge.get('train', [])
        test_inputs = challenge.get('test', [{}])
        if train_examples == []:
            continue
        examples_json = "\n".join([
            f"Example {i+1}:\n" + json.dumps(ex, indent=4)
            for i, ex in enumerate(train_examples)
        ])
        for i in range(len(test_inputs)):
            test_input = test_inputs[i]
            test_output = solution[i]
            if not test_input.get('input'):
                continue
            test_input_json = json.dumps(test_input.get('input', []), indent=4)
            final_prompt = ARC_PROMPT_TEMPLATE.format(
                examples=examples_json,
                test_input=test_input_json
            )
            challenge_id_new = challenge_id
            if(len(test_inputs)>1):
                challenge_id_new = f"{challenge_id}_test{i}"
            output = ARC_OUTPUT_TEMPLATE.format(output_grid=json.dumps(test_output, indent=4))
            prompts.append({
                "challenge_id": challenge_id_new,
                "prompt": final_prompt,
                "output": output
            })
    
    with open(save_path, 'w') as f:
        json.dump(prompts, f, indent=4)

In [ ]:
evaluation_prompts_path = os.path.join("../data/arc-agi-2025/prompts/arc-agi_evaluation_prompts.json")
generate_prompts_evaluation(evaluation_challenges, evaluation_solutions, evaluation_prompts_path)

In [19]:
import json


# Wrap every solution value in a dict under "output"
fixed = {k: {"output": v} for k, v in evaluation_solutions.items()}

with open("../data/arc-agi-2025/evaluation/arc-agi_evaluation_solutions_fixed.json", "w") as f:
    json.dump(fixed, f, indent=2)
